#**Human in the Loop Example**
This notebook will demonstater How user can modify the input in langgraph.

In [ ]:
!pip install langchain-core langchain-openai langgraph gradio langgraph-prebuilt


###**Retrive API key from Secrets and Set as an ENV**

In [ ]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
import os, getpass
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

# Arithmetic tools
def multiply(a: int, b: int) -> int:
    "Multiply"
    return a * b
def add(a: int, b: int) -> int:
    "Addition"
    return a + b
def divide(a: int, b: int) -> float:
    "Division"
    return a / b

tools = [add, multiply, divide]

# LLM + tool binding
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

# System prompt for assistant
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Human feedback node (no operation, just a breakpoint)
def human_feedback(state: MessagesState):
    pass

# Assistant node that calls LLM with tools
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("human_feedback", human_feedback)

# Control flow
builder.add_edge(START, "human_feedback")
builder.add_edge("human_feedback", "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# Memory checkpointing
memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

# Initial user input
initial_input = {"messages": [HumanMessage(content="Multiply 2 and 3")]}

# Thread id dictionary
thread = {"configurable": {"thread_id": "1"}}

# Run graph until interruption at human_feedback node
print("=== Running graph until interruption at human_feedback ===")
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()


state = graph.get_state(thread)  # returns StateSnapshot object

print("\n--- Current messages in state ---")
for msg in state.values['messages']:
    msg.pretty_print()


# Simulate human feedback: editing the conversation state
print("\n--- Applying human feedback: correcting input to multiply 3 and 3 ---")
graph.update_state(
    thread,
    {"messages": [HumanMessage(content="No, actually multiply 3 and 3!")]},
)

# Show updated state messages
new_state = graph.get_state(thread).values
for msg in new_state['messages']:
    msg.pretty_print()

# Resume graph execution from human_feedback node onwards
print("\n=== Resuming graph execution after human feedback ===")
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()


=== Running graph until interruption at human_feedback ===
================================ Human Message =================================

Multiply 2 and 3

--- Current messages in state ---
================================ Human Message =================================

Multiply 2 and 3

--- Applying human feedback: correcting input to multiply 3 and 3 ---
================================ Human Message =================================

Multiply 2 and 3
================================ Human Message =================================

No, actually multiply 3 and 3!

=== Resuming graph execution after human feedback ===
================================ Human Message =================================

No, actually multiply 3 and 3!
================================== Ai Message ==================================
Tool Calls:
  multiply (call_WPZTCRwHapU9wsjwFX9AqxpE)
 Call ID: call_WPZTCRwHapU9wsjwFX9AqxpE
  Args:
    a: 3
    b: 3
================================= Tool Message ========

##**Interactive Demo**

In [48]:
import os
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

# Arithmetic tools
def multiply(a: int, b: int) -> int:
    "Mul"
    return a * b
def add(a: int, b: int) -> int:
    "Add"
    return a + b
def divide(a: int, b: int) -> float:
    "Div"
    return a / b

tools = [add, multiply, divide]

# LLM + tool binding
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Human feedback node with confirmation
def human_feedback(state: MessagesState):
    last_user_msg = state["messages"][-1].content
    print(f"\nCurrent input: {last_user_msg}")
    choice = input("Do you want to update this input? (y/n): ").strip().lower()
    if choice == 'y':
        new_msg = input("Please enter your correction: ")
        # Update messages replacing last user input
        updated_messages = state["messages"][:-1] + [HumanMessage(content=new_msg)]
        return {"messages": updated_messages}
    else:
        # No change; return current state messages unchanged
        return {"messages": state["messages"]}

# Assistant node calling LLM with tools
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("human_feedback", human_feedback)

# Control flow
builder.add_edge(START, "human_feedback")
builder.add_edge("human_feedback", "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

# Memory checkpointing
memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

# Initial user input
initial_input = {"messages": [HumanMessage(content="Multiply 2 and 3")]}

thread = {"configurable": {"thread_id": "1"}}

# Run graph until interruption at human_feedback node
print("=== Running graph until interruption at human_feedback ===")
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

# Now resume the graph indefinitely (with user confirmation before update)
while True:
    # Run the graph from the human_feedback node onwards
    for event in graph.stream(None, thread, stream_mode="values"):
        event['messages'][-1].pretty_print()
    # Ask if user wants to continue or exit
    cont = input("\nDo you want to perform another calculation? (y/n): ").strip().lower()
    if cont != 'y':
        print("Exiting...")
        break
    else:
        # Prompt for new input explicitly, updating state before resuming
        user_input = input("Enter new arithmetic query: ")
        graph.update_state(thread, {"messages": [HumanMessage(content=user_input)]})


=== Running graph until interruption at human_feedback ===
================================ Human Message =================================

Multiply 2 and 3
================================ Human Message =================================

Multiply 2 and 3

Current input: Multiply 2 and 3
Do you want to update this input? (y/n): y
Please enter your correction: add 4 and 5
================================ Human Message =================================

add 4 and 5
================================== Ai Message ==================================
Tool Calls:
  multiply (call_fq31WpJI4vFXim5cR1hms244)
 Call ID: call_fq31WpJI4vFXim5cR1hms244
  Args:
    a: 2
    b: 3
  add (call_5tN7xAiAH0xWa2O2RQjwW7ah)
 Call ID: call_5tN7xAiAH0xWa2O2RQjwW7ah
  Args:
    a: 4
    b: 5
================================= Tool Message =================================
Name: add

9
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6, and the r